In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt


In [36]:
dataset_path = keras.utils.get_file("GOOG.csv", "http://localhost:8000/data/daily/GOOG.csv")

column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
raw_dataset = pd.read_csv(dataset_path, 
#                           names=column_names, 
                          dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                          header=0,
                          na_values = "?", 
                          comment='\t',
                          sep=",", 
                          skipinitialspace=True)



dataset = raw_dataset.copy()
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

278528/262569 [===============================] - 0s 0us/step


In [37]:
train_dataset.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
1196,2009-05-20,199.745850,201.524277,196.223755,197.306717,197.306717,4597600.0
3373,2018-01-11,1106.300049,1106.525024,1099.589966,1105.520020,1105.520020,978300.0
2615,2015-01-08,495.264526,500.724487,488.312805,499.928864,499.928864,3353600.0
3241,2017-07-05,901.760010,914.510010,898.500000,911.710022,911.710022,1813900.0
1504,2010-08-10,249.551910,251.365112,247.674118,250.227509,250.227509,4174800.0


In [38]:
train_stats = train_dataset.describe()
train_stats.pop("Close")
train_stats = train_stats.transpose()
train_stats



,count,mean,std,min,25%,50%,75%,max
Open,2827.0,4.254943e+02,2.803489e+02,49.274517,2.296141e+02,3.008085e+02,5.703768e+02,1.271000e+03
High,2827.0,4.292848e+02,2.823376e+02,50.541279,2.321824e+02,3.036997e+02,5.746403e+02,1.273890e+03
Low,2827.0,4.214477e+02,2.783615e+02,49.150326,2.280766e+02,2.981210e+02,5.644151e+02,1.246010e+03
Adj Close,2827.0,4.254501e+02,2.805135e+02,49.681866,2.300488e+02,3.010966e+02,5.701239e+02,1.249300e+03
Volume,2827.0,7.669548e+06,8.269812e+06,7900.000000,2.121800e+06,4.883500e+06,1.012730e+07,8.276810e+07


In [40]:
train_labels = train_dataset.pop('Close')
test_labels = test_dataset.pop('Close')

In [46]:
# test_dataset['High'] = map(test_dataset['High'])



0         51.693783
3         55.439419
7         52.404160
21        60.407108
24        60.918781
25        61.649033
26        60.049435
63        88.176498
67        84.863045
84        89.666801
86        93.332962
91        96.134735
93        99.294189
94       101.162041
99        98.409935
100       98.216202
112       96.720924
126       99.229607
130       94.311592
136       93.019997
146       89.617126
151       91.802910
153       90.138733
160       96.919632
163       96.532150
164       96.552025
166       93.332962
168       99.602180
174      110.322456
176      111.127228
           ...     
3360    1069.329956
3361    1064.199951
3363    1058.369995
3368    1093.569946
3370    1111.270020
3384    1186.890015
3404    1144.040039
3411    1127.599976
3412    1160.800049
3415    1158.589966
3420    1106.300049
3427    1034.800049
3431    1031.420044
3447    1037.000000
3455    1100.439941
3457    1110.750000
3468    1069.209961
3474    1135.819946
3478    1146.500000


In [41]:
#train_stats['mean']['High']
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

ValueError: Cannot broadcast operands together.